In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

The mlflow.client module provides a Python CRUD interface to MLflow Experiments, Runs, Model Versions, and Registered Models. This is a lower level API that directly translates to MLflow REST API calls. For a higher level API for managing an “active run”, use the mlflow module.

In [ ]:
client.create_experiment(name="experiment_from_the_client")

In [3]:
# WE CAN ALSO QUERY THE LIST OF MLRUNS IN EXPERIMENT #1 WHICH ARE ONLY ACTIVE USING
# ASCENDING RMSE VALUE
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [4]:
for run in runs:
    print(f"Run Id: {run.info.run_id}, RMSE: {run.data.metrics['rmse']:.4f}")

Run Id: be058c97b4484889bafa2f680f3170dd, RMSE: 12.2126
Run Id: f851d011ee574349a58c9bc527ee3bab, RMSE: 12.2126
Run Id: 546917c34ed548c5a1df16d2f9e6fa0d, RMSE: 12.2126


KeyError: 'rmse'

In [5]:
model_name = "NYC-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

In [6]:
latest_versions

[<ModelVersion: aliases=[], creation_timestamp=1685611470984, current_stage='Staging', description='', last_updated_timestamp=1685611820035, name='NYC-taxi-regressor', run_id='3b46d6e37c624b39b5385ab216341bd4', run_link='', source='/home/mikygetyos/Documents/My_Projects/NYC-Taxi-Duration/notebooks-exp-tracking/mlruns/1/3b46d6e37c624b39b5385ab216341bd4/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>,
 <ModelVersion: aliases=[], creation_timestamp=1685611492411, current_stage='Production', description='The model version 2 was transitioned to Production on 2023-06-01', last_updated_timestamp=1685616103298, name='NYC-taxi-regressor', run_id='a54494d120674c118b9d045875c79116', run_link='', source='/home/mikygetyos/Documents/My_Projects/NYC-Taxi-Duration/notebooks-exp-tracking/mlruns/1/a54494d120674c118b9d045875c79116/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>]

In [7]:
for version in latest_versions:
    print(f"version: {version.version} stage: {version.current_stage}")

version: 1 stage: Staging
version: 2 stage: Production


In [8]:
model_version = "2"
new_stage = "Production"
model_name = "NYC-taxi-regressor"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685611492411, current_stage='Production', description='The model version 2 was transitioned to Production on 2023-06-01', last_updated_timestamp=1685642692426, name='NYC-taxi-regressor', run_id='a54494d120674c118b9d045875c79116', run_link='', source='/home/mikygetyos/Documents/My_Projects/NYC-Taxi-Duration/notebooks-exp-tracking/mlruns/1/a54494d120674c118b9d045875c79116/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [9]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685611492411, current_stage='Production', description='The model version 2 was transitioned to Production on 2023-06-01', last_updated_timestamp=1685642694719, name='NYC-taxi-regressor', run_id='a54494d120674c118b9d045875c79116', run_link='', source='/home/mikygetyos/Documents/My_Projects/NYC-Taxi-Duration/notebooks-exp-tracking/mlruns/1/a54494d120674c118b9d045875c79116/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>

MODULAR IMPLEMENTATION OF NYC-TAXI-DURATION PREDICTION

In [10]:
import mlflow

In [11]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PULocationID', 'DOLocationID']
    numerical = 'trip_distance'
    train_dicts = df[categorical + numerical].to_dict(orient='reversed')
    return dv.transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [12]:
df = read_dataframe('./data/green_tripdata_2021-03.parquet')

In [13]:
run_id = '3b46d6e37c624b39b5385ab216341bd4'
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/tmp/ipykernel_124833/3729386333.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/home/mikygetyos/Documents/My_Projects/NYC-Taxi-Duration/notebooks-exp-tracking/preprocessor'

In [14]:
import pickle

with open("./preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [15]:
X_text = preprocess(df, dv)

TypeError: can only concatenate list (not "str") to list